In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import config
import datetime
from datetime import timedelta

# Urls con api news y categorías con Beautiful Soup
El resultado es una lista "seleccionadas" con 5 notas por categoría

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
#'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

In [ ]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

In [ ]:
from random import choice
headers = {'User-Agent':choice(desktop_agents)}
headers

In [ ]:
mydate = datetime.date(2018,9, 23) 
fecha = mydate.strftime("%Y-%m-%d")
fecha

In [ ]:
newsapi_key = config.newsAPI_KEY

In [ ]:
def obtener_urls(fecha):
    seleccionadas = []
    política, economía, elmundo, deportes, espectáculos, sociedad = 0,0,0,0,0,0
    desde= fecha
    hasta= fecha
    #print(fecha)
    fuente= 'la-nacion'
    pagina = 1
    print('Buscando notas')
    #while hasta que se completen los contadores
    while (política < 5) | (economía < 5) | (elmundo < 5) | (deportes < 5) | (espectáculos < 5) | (sociedad < 5):
        url = 'https://newsapi.org/v2/everything?sources=' + fuente + '&from=' + desde + '&to=' + hasta + '&page=' + str(pagina) +'&apiKey=' + newsapi_key
        pagina +=1

        try: 
            response = requests.get(url)
        except:
            print('no anduvo la api')
            break
        data = json.loads(response.text)
        if data['articles'] == []:
            print('no hay más')
            break

        #for todas la noticias de la pagina
        for i, nota in enumerate(data['articles']):
            response = requests.get(nota['url'])
            html_soup = BeautifulSoup(response.text, 'html.parser')
            for x in html_soup.find_all("strong", {"class": "categoria"}):
                categoria = x.find_all('a')[0].text.replace('\r\n', '').strip()

            #print(categoria, end = ', ')

            if (categoria == 'Política') & (política < 5):
                política +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Economía') & (economía <5):
                economía +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'El Mundo') & (elmundo <5):
                elmundo +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Deportes') & (deportes <5):
                deportes +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Espectáculos') & (espectáculos <5):
                espectáculos +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Sociedad') & (sociedad <5):
                sociedad +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])

    print('Cantidad de notas encontradas por categoría:', end = ' ' )
    print(política, economía, elmundo, deportes, espectáculos, sociedad)
    return seleccionadas
#que pasa si con todas las paginas que se recuperan de la api no se llega a llenar la cuota? puede pasar?


In [ ]:
def obtener_notas(seleccionadas):
    notas = []
    for nota in seleccionadas:
        print ('.', end ='')
        response = requests.get(nota['url'])
        soup = BeautifulSoup(response.text, 'html.parser')
        titular = soup.find('h1', {'class': 'titulo'}).text
        try:
            autor = soup.find('section', {'class': 'autor'}).find('a').text
        except:
            autor = ""
        fecha = nota['publishedAt'][8:10] + '/' + nota['publishedAt'][5:7] + '/' +nota['publishedAt'][:4]
        cuerpo = ''
        for x in soup.find('section', {'id': 'cuerpo'}).find_all('p'):
            cuerpo += x.text
        cuerpo = cuerpo.replace('\n','').replace('\r','')
        comentarios = not bool([True for x in soup.find_all('b') if "Debido a la sensibilidad" in x.text])
        #comentarios = bool(soup.find('div', {'class': 'ver-comentarios'}))
        
        notas.append({'titulo': titular,
                       'autor':autor,
                       'cuerpo':cuerpo,
                       'url': nota['url'],
                       'fecha':fecha,
                       'seccion': nota['categoria'],
                       'abierta_comentarios': comentarios})
    return notas


In [ ]:
%%time
notas = []
for dia in range(31):
    fecha = mydate.strftime("%Y-%m-%d")
    print('\n' + fecha)
    seleccionadas = obtener_urls(fecha)
    notas.extend(obtener_notas(seleccionadas))
    mydate += timedelta(days=1)

In [ ]:
import pandas as pd
df = pd.DataFrame(notas)

In [ ]:
df.fecha= pd.to_datetime(df["fecha"], dayfirst=True)

In [ ]:
#pd.DataFrame(notas).sort_values(by = ['fecha', 'seccion']).to_csv('./data/notas_lanacion.csv')
df.sort_values(by = ['fecha', 'seccion']).reset_index(drop= True).to_csv('./data/notas_lanacion.csv')

In [ ]:
df.shape

# Selenium para conseguir comments

In [44]:
import pandas as pd
df = pd.read_csv('./data/lanacion_notas.csv', index_col=0)

In [15]:
#df.abierta_comentarios.value_counts()

In [16]:
#Acá habría que hacerlo para toda la lista, pero pruebo sólo con la primera
#url = seleccionadas[0]['url']
#url = 'https://www.lanacion.com.ar/2181492-hugo-moyano-pedido-detencion-hijo'
#url = 'https://www.lanacion.com.ar/2183250-piden-juicio-oral-daniel-scioli-delitos-corrupcion'


In [17]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options


path_to_extension = r'C:\Users\p_bag\OneDrive\Documentos\1.17.0_0'
path_to_extension_linux = r'/home/pablo/Documentos/1.17.0_0'
windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
linux = '/home/pablo/Descargas/chromedriver'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)
chrome_options.add_argument("--incognito")

 


In [38]:
def scrapear(urls):
    comentarios= []
    #contador = 0
    #primera = True
    #chrome_options.add_argument('User-Agent :' + choice(desktop_agents))
    #driver = webdriver.Chrome(windows, chrome_options=chrome_options)
    #driver.create_options()
    for nota_id, url in zip(urls.index, urls.values):
        #que solo reabra el driver en divisores de 10?
        #if contador % 4 == 0:
        #    if not primera:
        #        driver.quit()
        #    driver = webdriver.Chrome(windows, chrome_options=chrome_options)
        #    driver.create_options()
        #    primera = False
        #contador+=1
        driver = webdriver.Chrome(windows, chrome_options=chrome_options)
        driver.create_options()
        driver.get(url)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'fyre-stream-more-container')))
            #print ("Hay boton")
            button = True
        except TimeoutException:
            #print ("No hay boton")
            button = False
        #Que pasa si la nota esta cerrada a comentarios
        while button:
            try:
                driver.find_element_by_class_name('fyre-stream-more-container').click()
            except: 
                button = False
        #todos los comentarios con me gusta, horario y autores,etc
        try:
            todo = driver.find_element_by_class_name('fyre-stream-content')
            a= todo.find_elements_by_class_name('fyre-comment-article')
            for item in a:
                try:
                    comentarios.append({'texto': item.find_elements_by_class_name("fyre-comment")[0].text,
                                        'usuario' : item.find_elements_by_class_name("fyre-comment-username")[0].text,
                                        'likes' : item.find_elements_by_class_name("fyre-comment-like-count")[0].text,
                                        'fecha' : item.find_elements_by_class_name("fyre-comment-date")[0].text,
                                        'respuestas' : len(item.find_elements_by_class_name('fyre-comment')) -1,
                                        'nota_id': nota_id,
                                        'nota_url': url
                                       })
                    print('.', end = '')
                except:
                    pass
        except:
            comentarios.append({'texto': "NOTA SIN COMENTARIOS",
                                        'usuario' : "",
                                        'likes' : "",
                                        'fecha' : "",
                                        'respuestas' : -1,
                                        'nota_id': nota_id,
                                        'nota_url': url
                                       })
        driver.quit()
    return comentarios

In [19]:
#ser trabo en este
#scrapear(df.url[10:14])
#resultados = scrapear(pd.Series(['https://www.lanacion.com.ar/2183681-una-socia-viuda-munoz-se-convirtio-primera']))

In [45]:
df = df[400:-100]

In [46]:
corte = df.url.shape[0]//4
lista_urls = []
lista_urls.append(df.url[:corte])
lista_urls.append(df.url[corte:corte*2])
lista_urls.append(df.url[corte*2:corte*3])
lista_urls.append(df.url[corte*3:df.shape[0]])

In [47]:
%%time
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1)(delayed(scrapear)(urls) for urls in lista_urls)

Wall time: 1h 43min 18s


In [48]:
comentarios = results[0] + results[1] + results[2] + results[3]

In [49]:
import pandas as pd
df_comentarios = pd.DataFrame(comentarios)
df_comentarios = df_comentarios.dropna().reset_index(drop = True)
df_comentarios.loc[:, 'respuestas'] = df_comentarios.respuestas.astype(int)
df_comentarios['es_respuesta'] = df_comentarios.texto.str.contains('^@')
df_comentarios

,fecha,likes,nota_id,nota_url,respuestas,texto,usuario,es_respuesta
0,,,400,https://www.lanacion.com.ar/2179112-para-comba...,-1,NOTA SIN COMENTARIOS,,False
1,16:04 09/10/2018,,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,No se olviden que Bolsonaro va a gobernar con ...,obvius,False
2,11:45 09/10/2018,1,402,https://www.lanacion.com.ar/2179487-para-ameri...,1,Tenemos que hacer una union comercial integral...,Richard43,False
3,15:51 09/10/2018,1,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,@Richard43 Este señor dijo que no le importa e...,obvius,True
4,11:40 09/10/2018,3,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,poco a poco y paso a paso la izquierda muestra...,Richard43,False
5,11:29 09/10/2018,,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,"Pobre Brasil,sigue el camino de Argentina !",betunico,False
6,22:32 08/10/2018,1,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,que periodico de izquierda.. fascistas son ud LN,ezequiel_inver,False
7,20:20 08/10/2018,4,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,Jair Bolsonaro no quiere que Brasil sea la nue...,omarhess,False
8,20:01 08/10/2018,6,402,https://www.lanacion.com.ar/2179487-para-ameri...,0,¡¡¡Qué cosa extraña sucede con la prensa canal...,Morera,False
9,19:42 08/10/2018,2,402,https://www.lanacion.com.ar/2179487-para-ameri...,3,Bolsonaro es un infeliz como también lo es Had...,valeriov,False


In [59]:
#df_comentarios.to_csv('./data/commentarios_lanacion.csv')

#versión sin conteo de respuestas
%%time
textos = todo.find_elements_by_class_name('fyre-comment')
users = todo.find_elements_by_class_name("fyre-comment-username")
fechas = todo.find_elements_by_class_name("fyre-comment-date")
likes = todo.find_elements_by_class_name("fyre-comment-like-count")
usuarios =[x.text for x in users]
textos = [x.text for x in textos]
fechas = [x.text for x in fechas]
likes = [x.text for x in likes]
import pandas  as pd
df = pd.DataFrame({'usuario': usuarios,
                     'comment': textos,
                   'fecha': fechas,
                   'likes': likes
                  })

In [ ]:
#https://www.lanacion.com.ar/2183896-hora-cambios-relacion-estado-iglesia, esta nota no abre automatico los comentarios
#El boton qeu abre lo comentarios es el ver comentarios
<div class="ver-comentarios ver-comentarios">
    <a href="#comentarios">Ver comentarios <span class="ds-none livefyre-commentcount" data-lf-site-id="356483" data-lf-article-id="1353322">218</span></a>
</div>
#en algunas aparece el boton y te lleva a los comentarios que estan abajo,
# en esa el boton dice 218 comments pero te lleva a la cartel de nota cerrada a comentarios
#hay notas cerradas a comentarios que te los muestra y otras que no te lo muestra?

#ejemplo de nota cerrada a comentarios:
#https://www.lanacion.com.ar/688808-cambios-en-un-area-sensible
#hay notas cerradas que tiene comentarios

In [ ]:
driver = webdriver.Chrome(windows, chrome_options=chrome_options)


In [ ]:
driver.quit()